## Step 1: Install required packages

In [22]:
!pip install sqlalchemy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [23]:
!pip install psycopg2-binary sqlalchemy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [25]:
from sqlalchemy import create_engine
import pandas as pd

## Step 2: Set up PostgreSQL database connection info

In [29]:
username = 'postgres'      
password = '123'     
host = 'localhost'              
port = '5432'                   
database = '5310GP'      

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

## Step 3: Load CSV file and preprocess

In [32]:
df = pd.read_csv("supermarket_sales - Sheet1.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('%', 'percent')
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
df['time'] = pd.to_datetime(df['time']).dt.time

/var/folders/rb/d77z3j_56wn8cd0292h31j740000gn/T/ipykernel_5707/1356442537.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time']).dt.time


## Step 4: Create surrogate IDs for normalization

In [35]:
df['product_line_id'] = df['product_line'].astype('category').cat.codes + 1
df['city_id'] = df['city'].astype('category').cat.codes + 1
df['store_id'] = df['branch'].astype('category').cat.codes + 1
df['customer_id'] = df.index + 1  # Assume each row represents one customer
df['payment_id'] = df['payment'].astype('category').cat.codes + 1

# Generate product_id based on unique combination of product line and unit price
product_combos = df[['product_line_id', 'unit_price']].drop_duplicates().reset_index(drop=True)
product_combos['product_id'] = range(1, len(product_combos) + 1)
df = df.merge(product_combos, on=['product_line_id', 'unit_price'], how='left')


## Step 5: Create 10 normalized tables

In [38]:
# Products table
products = product_combos[['product_id', 'product_line_id', 'unit_price']]

# Product lines
product_lines = df[['product_line_id', 'product_line']].drop_duplicates().rename(columns={'product_line': 'product_line_name'})

# Cities
cities = df[['city_id', 'city']].drop_duplicates().rename(columns={'city': 'city_name'})

# Stores
stores = df[['store_id', 'branch', 'city_id']].drop_duplicates().rename(columns={'branch': 'branch_code'})

# Customers
customers = df[['customer_id', 'gender', 'customer_type']]

# Payments
payments = df[['payment_id', 'payment']].drop_duplicates().rename(columns={'payment': 'payment_method'})

# Calendar date dimension
calendar = df[['date']].drop_duplicates().rename(columns={'date': 'transaction_date'})
calendar['day'] = calendar['transaction_date'].dt.day
calendar['month'] = calendar['transaction_date'].dt.month
calendar['year'] = calendar['transaction_date'].dt.year
calendar['quarter'] = calendar['transaction_date'].dt.quarter
calendar['weekday'] = calendar['transaction_date'].dt.day_name()

# Sales transaction header table
sales_transactions = df[['invoice_id', 'store_id', 'customer_id', 'date', 'time', 'payment_id', 'total']].rename(columns={
    'date': 'transaction_date',
    'time': 'transaction_time',
    'payment_id': 'payment_method',
    'total': 'total_amount'
})

# Sales item detail table
sales_items = df[['invoice_id', 'product_id', 'unit_price', 'quantity']]

# Financials (revenue & cost breakdown)
financials = df[['invoice_id', 'tax_5percent', 'cogs', 'gross_income', 'rating']].rename(columns={'tax_5percent': 'tax'})


## Step 6: Upload all tables to PostgreSQL

In [41]:
tables = {
    'products': products,
    'product_lines': product_lines,
    'cities': cities,
    'stores': stores,
    'customers': customers,
    'payments': payments,
    'calendar': calendar,
    'sales_transactions': sales_transactions,
    'sales_items': sales_items,
    'financials': financials
}

# Loop through and upload each table
for table_name, df_table in tables.items():
    df_table.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"✅ Table '{table_name}' uploaded successfully.")

✅ Table 'products' uploaded successfully.
✅ Table 'product_lines' uploaded successfully.
✅ Table 'cities' uploaded successfully.
✅ Table 'stores' uploaded successfully.
✅ Table 'customers' uploaded successfully.
✅ Table 'payments' uploaded successfully.
✅ Table 'calendar' uploaded successfully.
✅ Table 'sales_transactions' uploaded successfully.
✅ Table 'sales_items' uploaded successfully.
✅ Table 'financials' uploaded successfully.


## Step 7: Add Primary and Foreign Key Constraints (PostgreSQL)

Since the initial tables were created using `pandas.DataFrame.to_sql()`, which does not include primary key or foreign key constraints by default, we manually add them afterward using SQL `ALTER TABLE` statements in pgAdmin.

Below is the SQL code to define primary keys and foreign keys for the 10 normalized tables:


### Add primary key in pgadmin
``` sql
ALTER TABLE products ADD PRIMARY KEY (product_id);
ALTER TABLE product_lines ADD PRIMARY KEY (product_line_id);
ALTER TABLE cities ADD PRIMARY KEY (city_id);
ALTER TABLE stores ADD PRIMARY KEY (store_id);
ALTER TABLE customers ADD PRIMARY KEY (customer_id);
ALTER TABLE payments ADD PRIMARY KEY (payment_id);
ALTER TABLE calendar ADD PRIMARY KEY (transaction_date);
ALTER TABLE sales_transactions ADD PRIMARY KEY (invoice_id);
ALTER TABLE financials ADD PRIMARY KEY (invoice_id);
ALTER TABLE sales_items ADD PRIMARY KEY (invoice_id, product_id);

### Add foreign key in pgadmin
``` sql
ALTER TABLE sales_transactions
ADD CONSTRAINT fk_sales_transactions_payment
FOREIGN KEY (payment_method) REFERENCES payments(payment_id);

ALTER TABLE sales_transactions
ADD CONSTRAINT fk_sales_transactions_store
FOREIGN KEY (store_id) REFERENCES stores(store_id);

ALTER TABLE sales_transactions
ADD CONSTRAINT fk_sales_transactions_customer
FOREIGN KEY (customer_id) REFERENCES customers(customer_id);

ALTER TABLE stores
ADD CONSTRAINT fk_stores_city
FOREIGN KEY (city_id) REFERENCES cities(city_id);

ALTER TABLE products
ADD CONSTRAINT fk_products_product_line
FOREIGN KEY (product_line_id) REFERENCES product_lines(product_line_id);

ALTER TABLE sales_items
ADD CONSTRAINT fk_sales_items_invoice
FOREIGN KEY (invoice_id) REFERENCES sales_transactions(invoice_id);

ALTER TABLE sales_items
ADD CONSTRAINT fk_sales_items_product
FOREIGN KEY (product_id) REFERENCES products(product_id);

ALTER TABLE financials
ADD CONSTRAINT fk_financials_invoice
FOREIGN KEY (invoice_id) REFERENCES sales_transactions(invoice_id);

ALTER TABLE sales_transactions
ADD CONSTRAINT fk_sales_transactions_date
FOREIGN KEY (transaction_date) REFERENCES calendar(transaction_date);